In [2]:
# This file costructs surrogate models for the input datasets
import numpy as np   
import pandas as pd
import os
import shutil
import json
import math
import time
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Torch specific module imports
import torch
import gpytorch 

# botorch specific modules
from botorch.fit import fit_gpytorch_model
from botorch.models.gpytorch import GPyTorchModel
from botorch.optim import optimize_acqf, optimize_acqf_discrete
from botorch import fit_gpytorch_mll
from botorch.acquisition.monte_carlo import (
    qExpectedImprovement,
    qNoisyExpectedImprovement,
)
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.acquisition import UpperConfidenceBound, ExpectedImprovement

# Plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Tick parameters
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1

plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['legend.fontsize'] = 15

# User defined python classes and files
import input_class 
import code_inputs as model_input
import utils_dataset as utilsd
import surrogate_models

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
bounds = torch.tensor([[-10.0], [12.0]])

batch_size = 1
num_restarts= 10 
raw_samples = 512

def optimize_acqf_and_get_observation(acq_func, X_test, Y_test):
    """Optimizes the acquisition function, and returns a new candidate"""
    # optimize
    candidates, _ = optimize_acqf_discrete(
        acq_function=acq_func,
        choices=X_test,
        q=batch_size,
        max_batch_size=2048,
        num_restarts=num_restarts,
        raw_samples=raw_samples,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
        unique=True
    )
    
    # observe new values
    new_x = candidates.detach()
    b = [1 if torch.all(X_test[i].eq(new_x)) else 0 for i in range(0,X_test.shape[0]) ]
    b = torch.tensor(b).to(torch.int)
    index = b.nonzero()[0][0]
    new_y = torch.reshape(Y_test[0,index],(1,1))
    
    X_test_new = X_test[torch.arange(0, X_test.shape[0]) != index, ...]
    Y_test_new = Y_test[..., torch.arange(0, Y_test.shape[1]) != index]
    
    return new_x, new_y, index, X_test_new, Y_test_new

In [ ]:
warnings.filterwarnings("ignore", category=BadInitialCandidatesWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Create a new directory if it does not exist
isExist = os.path.exists(model_input.output_folder)
if not isExist:
    os.makedirs(model_input.output_folder)
    print("The new directory is created!", model_input.output_folder)
    
# Copy input parameters file to output folder
shutil.copy2('surrogate_model_inputs.py',model_input.output_folder)
# Copy surrogate model file to output folder
shutil.copy2('surrogate_models.py',model_input.output_folder)

# BO Trials
n_trials = model_input.n_trials
n_update = model_input.n_update
verbose = model_input.verbose

test_size = model_input.test_size
train_GP = model_input.train_GP

GP_0 = model_input.GP_0_BO


num_nodes = model_input.num_nodes
saveModel_filename = model_input.saveModel_filename
    
best_observed_all_ei0 = []

# Average over multiple trials
for trial in range(1, n_trials + 1):
    t0 = time.monotonic()
    if model_input.random_seed == 'time':
        random_seed = int(t0)
    elif model_input.random_seed == 'iteration':
        random_seed = trial
        
    print(f"\n -------------------- Trial {trial:>2} of {n_trials} --------------------\n", end="")
    best_observed0 = []

    # Getting initial data and fitting models with initial data
    if model_input.standardize_data:
        X_train, X_test, Y_train, Y_test, Var_train, Var_test, scalerX_transform, scalerY_transform = utilsd.generate_training_data(random_seed,model_input.test_size)
    else:
        X_train, X_test, Y_train, Y_test, Var_train, Var_test = utilsd.generate_training_data(random_seed,model_input.test_size)
        
    # Finding best value in initial data
    if model_input.maximization:
        best_observed_value = Y_train.max()
        optimal_solution = torch.cat([Y_train[0],Y_test[0]]).max()
    else:
        best_observed_value = Y_train.min()
        optimal_solution = torch.cat([Y_train[0],Y_test[0]]).min()
    
    # If optimal value is present in the initial dataset sample remove it  
    if (best_observed_value.eq(optimal_solution)) and model_input.maximization:
        print('Max in training set, removing it before training models.')
        optimal_position = torch.argmax(Y_train)
        
        # Add max value to test/exploration set
        X_add_toTest = torch.reshape(X_train[optimal_position,:],(1,X_train.shape[1]))
        X_test = torch.cat([X_test,X_add_toTest])
        Y_add_toTest = torch.reshape(optimal_solution,(1,1))      
        Y_test = torch.cat((Y_test,Y_add_toTest),1)
        
        # Remove max value from training set
        X_train = X_train[torch.arange(0, X_train.shape[0]) != optimal_position, ...]
        Y_train = Y_train[..., torch.arange(0, Y_train.shape[1]) != optimal_position]
        
        # Update best observed value
        best_observed_value = Y_train.max()
        
    elif (best_observed_value.eq(optimal_solution)) and not model_input.maximization:
        print('Min in training set, removing it before training models.')
        optimal_position = torch.argmin(Y_train)
        
        # Add min value to test/exploration set
        X_add_toTest = torch.reshape(X_train[optimal_position,:],(1,X_train.shape[1]))
        X_test = torch.cat([X_test,X_add_toTest])
        Y_add_toTest = torch.reshape(optimal_solution,(1,1))      
        Y_test = torch.cat((Y_test,Y_add_toTest),1)
        
        # Remove min value from training set
        X_train = X_train[torch.arange(0, X_train.shape[0]) != optimal_position, ...]
        Y_train = Y_train[..., torch.arange(0, Y_train.shape[1]) != optimal_position]
        
        # Update best observed value
        best_observed_value = Y_train.min()
    
    # Initialize data for training gp-0 and gp-l models
    X_train0, Y_train0, X_test0, Y_test0 = X_train, Y_train, X_test, Y_test
            
    n_batch = model_input.n_batch_perTrial
    
    # Initialize likelihood, GP model and acquisition function for the models
    #--------------------------- GP-0 ---------------------------#
    if GP_0:
        likelihood_gp0 = gpytorch.likelihoods.GaussianLikelihood()
        model_gp0 = surrogate_models.ExactGPModel(X_train0, Y_train0, likelihood_gp0)           
        # AcqFunc_0 = UpperConfidenceBound(model_gp0, beta=0.1) 
        AcqFunc_0 = ExpectedImprovement(model=model_gp0, best_f=best_observed_value, maximize=model_input.maximization)
        best_observed0.append(best_observed_value)  # Appending to best_observed list for the given trial
       
    # run N_BATCH rounds of BayesOpt after the initial random batch
    for iteration in range(1, n_batch + 1):

        if GP_0:
            if ((iteration-1)%n_update==0):
                # fit the models every 10 iterations
                model_gp0, likelihood_gp0 = surrogate_models.train_surrogate_gp0(saveModel_filename, test_size, num_nodes, X_train0, Y_train0)
            
            # optimize and get new observation using acquisition function
            new_x0, new_y0, index, X_test_new0, Y_test_new0 = optimize_acqf_and_get_observation(AcqFunc_0, X_test0, Y_test0)
            
            # Update remaining choices tensor
            X_test0 = X_test_new0
            Y_test0 = Y_test_new0

            # Update training points
            X_train0 = torch.cat([X_train0, new_x0])
            Y_train0 = torch.cat([Y_train0[0], new_y0[0]])
            Y_train0 = torch.reshape(Y_train0,(1,Y_train0.shape[0]))

            # update progress
            if model_input.maximization:
                best_value_ei0 = Y_train0.max()
            elif not model_input.maximization:
                best_value_ei0 = Y_train0.min()
            best_observed0.append(best_value_ei0)

            # AcqFunc_0 = UpperConfidenceBound(model_gp0, beta=0.1) 
            AcqFunc_0 = ExpectedImprovement(model=model_gp0, best_f=best_value_ei0, maximize=model_input.maximization)
      
        if verbose:
            print(
                f"\nBatch {iteration:>2}: best_value (GP-0, GP-Linear, GP-NN) = ",
                f"({best_value_ei0:>4.2f}, {best_value_eiL:>4.2f}, {best_value_eiNN:>4.2f})",
                end="",)

    t1 = time.monotonic()
    
    print(f"time = {t1-t0:>4.2f}.")
    # Appending to common list of best observed values, with number of rows equal to number of trials
    if GP_0:
        best_observed_all_ei0.append(best_observed0)       

